In [ ]:
from requests_html import HTMLSession

session = HTMLSession()
portatil_links = []

for i in range(1,3):
    print(f'Pagina {i}')
    r = session.get(url=f"https://www.pcbox.com/ordenadores/ordenadores-portatiles?page={i}")
    
    articles = r.html.find('article')
    links = [article.find('a', first=True).attrs['href'] for article in articles]
    portatil_links.extend(links)

In [1]:
import sys
import json
import requests
from bs4 import BeautifulSoup

def scrape_laptop_specs(url):
    # Realiza la petición HTTP
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error al obtener la página. Código HTTP: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    
    # Diccionario donde se guardarán las secciones completas de la tabla
    sections = {}

    # Se buscan todas las tablas con la clase utilizada en las especificaciones
    tables = soup.find_all("table", class_="vtex-table-description-extended_carac")
    
    # Iteramos sobre cada tabla (por ejemplo, puede haber una en cada columna)
    for table in tables:
        current_section = None
        # Iteramos sobre las filas que contienen la clase indicativa
        for row in table.find_all("tr", class_="vtex-table-description-row"):
            # Si la fila tiene un <td> con colspan="2" y un div de título, es un encabezado de sección
            header_td = row.find("td", colspan="2")
            if header_td:
                title_div = header_td.find("div", class_="vtex-table-description-title")
                if title_div:
                    section_title = title_div.get_text(strip=True)
                    current_section = section_title
                    sections[current_section] = {}
                    continue  # pasamos a la siguiente fila

            # Si la fila contiene clave y valor, los extraemos
            key_td = row.find("td", class_="vtex-table-description-key")
            value_td = row.find("td", class_="vtex-table-description-value")
            if key_td and value_td and current_section:
                key = key_td.get_text(strip=True)
                value = value_td.get_text(strip=True)
                sections[current_section][key] = value

    # Mapeamos las secciones extraídas a los campos requeridos:
    # - Procesador → sección "Procesador"
    # - RAM → sección "Memoria"
    # - Almacenamiento → sección "Medios de almacenaje"
    # - Graficos → sección "Gráficos"
    # - Pantalla → sección "Exhibición"
    # - Sistema Operativo → sección "Software"
    # - Bateria → sección "Batería"
    mapping = {
        "Procesador": "Procesador",
        "Memoria": "RAM",
        "Medios de almacenaje": "Almacenamiento",
        "Gráficos": "Graficos",
        "Exhibición": "Pantalla",
        "Software": "Sistema Operativo",
        "Batería": "Bateria"
    }

    result = {}
    for seccion_original, campo in mapping.items():
        # Si la sección existe, se agrega; de lo contrario se deja vacía.
        result[campo] = sections.get(seccion_original, {})

    return result

In [6]:
url = "https://www.pcbox.com/82fg01r1sp-ip-s500-i5-1135g7-8gb-512gb/p"
specs = scrape_laptop_specs(url)
if specs is not None:
    specs_json = json.dumps(specs, indent=4, ensure_ascii=False)

In [11]:
print(specs_json)

{
    "Procesador": {
        "Frecuencia del procesador": "2,4 GHz",
        "Fabricante de procesador": "Intel",
        "Frecuencia del procesador turbo": "4,2 GHz",
        "Modelo del procesador": "i5-1135G7",
        "Número de núcleos de procesador": "4",
        "Chipset": "Intel® SoC",
        "Familia de procesador": "Intel® Core™ i5",
        "Potencia de diseño térmico configurable-baja": "12 W",
        "Frecuencia de potencia de diseño térmico configurable-baja": "0,9 GHz",
        "Frecuencia de potencia de diseño térmico configurable-alta": "2,4 GHz",
        "Potencia de diseño térmico configurable-alta": "28 W",
        "Generación del procesador": "Intel® Core™ i5 de 11ma Generación"
    },
    "RAM": {
        "Forma de factor de memoria": "Incorporado",
        "Velocidad de memoria del reloj": "3200 MHz",
        "Memoria interna": "8 GB",
        "Tipo de memoria interna": "DDR4-SDRAM",
        "Memoria interna máxima": "8 GB"
    },
    "Almacenamiento": {
     

In [9]:
import csv
import re
import json

def simplify_procesador(proc_dict):
    fabricante = proc_dict.get("Fabricante de procesador", "").strip()
    familia = proc_dict.get("Familia de procesador", "").strip()
    modelo = proc_dict.get("Modelo del procesador", "").strip()
    # Eliminar símbolos de marca (® y ™)
    familia_clean = re.sub(r"[®™]", "", familia).strip()
    # Si el fabricante no está incluido, lo anteponemos
    if fabricante and fabricante not in familia_clean:
        familia_clean = f"{fabricante} {familia_clean}".strip()
    # Si el modelo empieza con el mismo token final de la familia, lo eliminamos para evitar duplicados
    last_token = familia_clean.split()[-1]
    if modelo.startswith(last_token):
        new_model = modelo[len(last_token):]
        # Si new_model comienza con guión, lo concatenamos directamente
        if new_model.startswith("-"):
            final = f"{familia_clean}{new_model}"
        else:
            final = f"{familia_clean}-{modelo}"
    else:
        final = f"{familia_clean}-{modelo}"
    return final

def simplify_ram(ram_dict):
    # Extrae el número de GB desde "Memoria interna" (por ejemplo, "8 GB")
    ram_value = ram_dict.get("Memoria interna", "")
    match = re.search(r"(\d+)", ram_value)
    return match.group(1) if match else ""

def simplify_tipo_ram(ram_dict):
    tipo_ram = ram_dict.get("Tipo de memoria interna", "")
    # Extrae la parte DDR (por ejemplo, "DDR4" de "DDR4-SDRAM")
    match = re.search(r"(DDR\d+)", tipo_ram, re.IGNORECASE)
    return match.group(1).upper() if match else ""

def simplify_almacenamiento(alm_dict):
    # Se prefiere "Capacidad total de SSD", pero si no existe se usa "SDD, capacidad"
    valor = alm_dict.get("Capacidad total de SSD", "") or alm_dict.get("SDD, capacidad", "")
    match = re.search(r"(\d+)", valor)
    return match.group(1) if match else ""

def simplify_graficos(graf_dict):
    return graf_dict.get("Modelo de adaptador gráfico incorporado", "").strip()

def simplify_pantalla(pant_dict):
    diag = pant_dict.get("Diagonal de la pantalla", "")
    # Se extrae la medida en pulgadas que aparece entre paréntesis, por ejemplo: (15.6")
    match = re.search(r"\(([\d\.]+)\"", diag)
    return match.group(1) if match else ""

def simplify_resolucion(pant_dict):
    resol = pant_dict.get("Resolución de la pantalla", "")
    # Se buscan los dos números (ancho y alto)
    numbers = re.findall(r"(\d+)", resol)
    if len(numbers) >= 2:
        return f"{numbers[0]}x{numbers[1]}"
    return ""

def simplify_sistema(soft_dict):
    return soft_dict.get("Sistema operativo instalado", "").strip()

def simplify_bateria(bat_dict):
    bat = bat_dict.get("Capacidad de batería", "")
    match = re.search(r"(\d+)", bat)
    return match.group(1) if match else ""

def simplify_specs(specs):
    simplified = {}
    simplified["Procesador"] = simplify_procesador(specs.get("Procesador", {}))
    simplified["RAM"] = simplify_ram(specs.get("RAM", {}))
    simplified["Tipo RAM"] = simplify_tipo_ram(specs.get("RAM", {}))
    simplified["Almacenamiento"] = simplify_almacenamiento(specs.get("Almacenamiento", {}))
    simplified["Graficos"] = simplify_graficos(specs.get("Graficos", {}))
    simplified["Pantalla"] = simplify_pantalla(specs.get("Pantalla", {}))
    simplified["Resolucion"] = simplify_resolucion(specs.get("Pantalla", {}))
    simplified["Sistema Operativo"] = simplify_sistema(specs.get("Sistema Operativo", {}))
    simplified["Bateria"] = simplify_bateria(specs.get("Bateria", {}))
    return simplified

In [ ]:
simplified = simplify_specs(specs)
    
# Definición de las columnas del CSV en el orden deseado
fieldnames = [
    "Procesador",
    "RAM",
    "Tipo RAM",
    "Almacenamiento",
    "Graficos",
    "Pantalla",
    "Resolucion",
    "Sistema Operativo",
    "Bateria"
]

# Generamos el CSV con una única fila (se puede extender para múltiples registros)
with open("specs_simplified.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow(simplified)

CSV generado: specs_simplified.csv
